COPYRIGHT © 2018 Kiran Arun <kironni@gmail.com>

### Setup

In [0]:
import os
!pip3 install gym 1>/dev/null
!pip3 install keras-rl 1>/dev/null

In [2]:
!rm -rf Reinforcement_Learning-101-challenge
!git clone https://github.com/arpat/Reinforcement_Learning-101-challenge.git 1>/dev/null
!bash Reinforcement_Learning-101-challenge/scripts/install_ngrok.sh 1>/dev/null
!bash Reinforcement_Learning-101-challenge/scripts/install_novnc.sh 1>/dev/null

Cloning into 'Reinforcement_Learning-101-challenge'...
remote: Counting objects: 59, done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 59 (delta 28), reused 37 (delta 13), pack-reused 0
Unpacking objects: 100% (59/59), done.


In [19]:
# DEFINITELY DO NOT EDIT THIS CELL

# Start noVNC remote Xwindows in a browser with TurboVNC
!bash /content/Reinforcement_Learning-101-challenge/scripts/kill-novnc.sh >/dev/null
!sleep 2
# start novnc server and expose via ngrok NOT HTTPS
os.environ['PATH'] += "${PATH}:/opt/VirtualGL/bin:/opt/TurboVNC/bin"
get_ipython().system_raw('/opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup > /content/.vnc/stdout 2>&1 &')
# ngrok remote tunnel
get_ipython().system_raw('/content/ngrok http 5901 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print('XWindow Link:', json.load(sys.stdin)['tunnels'][0]['public_url'])"
# get Xwindows going (right click to get openbox menu)
get_ipython().system_raw('DISPLAY=:1 openbox &')
# one-time-password
!grep "one-time password:" /content/.vnc/stdout

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
XWindow Link: https://8087f06b.ngrok.io
Full control one-time password: 41252877


In [0]:
# DO NOT EDIT THIS CELL

# set up Xvfb and attach it's own vncserver and test with spinning gears
!pkill Xvfb   # clean-up
!pkill x11vnc # clean-up

get_ipython().system_raw('DISPLAY=:1 xterm &')
get_ipython().system_raw('/usr/bin/Xvfb :99 -screen 0 640x480x24 &')
get_ipython().system_raw('/usr/bin/x11vnc -rfbport 5902 -forever -display :99 &')
!sleep 2
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 glxgears &')

In [0]:
# open a terminal into the glxgears session using TightVNC
#get_ipython().system_raw('DISPLAY=:99 xterm -e /bin/bash -l -c "pkill glxgears"')
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 xterm -geometry 105x35 &')

# RL Challenge

In [6]:
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import gym

import keras
import keras.layers as layers
from rl import agents,memory,policy

Using TensorFlow backend.


### Hyperparameters

In [0]:
# Hyperparams

# number of steps to keep in experience buffer
memory_limit = 400

# discount value
gamma = 0.99

# how much to update target graph
target_model_update = 1e-2

# learning rate
learning_rate = 1e-2

# number of steps to sample from buffer to train on
batch_size = 32

In [8]:
# DO NOT EDIT

# setting environment
env = gym.make('CartPole-v1')
num_actions = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


### Neural Network

In [9]:
keras.backend.clear_session()

# create neural net
model = keras.models.Sequential()

# keep this as input layer
model.add(layers.Flatten(input_shape=(1,) + env.observation_space.shape))

# keep this as output layer
model.add(layers.Dense(num_actions))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


### Optimizer

https://keras.io/optimizers/

In [0]:
# set optimizer
optimizer = keras.optimizers.SGD(lr=learning_rate)

### Memory

In [0]:
# DO NOT EDIT

# set experience buffer
memory = memory.SequentialMemory(limit=memory_limit,
                                    window_length=1)

### Policy

choose from: https://github.com/keras-rl/keras-rl/blob/master/rl/policy.py

[info on choosing](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-7-action-selection-strategies-for-exploration-d3a97b7cceaf)

In [0]:
# set policy
policy = policy.BoltzmannQPolicy()

### Agent

choose from: https://github.com/keras-rl/keras-rl/tree/master/rl/agents

and use: https://github.com/keras-rl/keras-rl/tree/master/examples

In [0]:
# create agent
model = agents.dqn.DQNAgent(model=model,
                            nb_actions=num_actions,
                            memory=memory,
                            gamma=gamma,
                            batch_size=batch_size,
                            nb_steps_warmup=100,
                            target_model_update=target_model_update,
                            policy=policy)

In [0]:
# DO NOT EDIT

# compile model
model.compile(optimizer,
            metrics=['mae'])

In [15]:
# DO NOT EDIT

# train
history = model.fit(env,
                  nb_steps=1000,
                  verbose=2)

Training for 1000 steps ...
  28/1000: episode: 1, duration: 0.278s, episode steps: 28, steps per second: 101, episode reward: 28.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.078 [-0.405, 0.901], loss: --, mean_absolute_error: --, mean_q: --
  39/1000: episode: 2, duration: 0.015s, episode steps: 11, steps per second: 710, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.455 [0.000, 1.000], mean observation: 0.124 [-0.800, 1.403], loss: --, mean_absolute_error: --, mean_q: --
  67/1000: episode: 3, duration: 0.044s, episode steps: 28, steps per second: 642, episode reward: 28.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.429 [0.000, 1.000], mean observation: 0.019 [-1.016, 1.562], loss: --, mean_absolute_error: --, mean_q: --
  82/1000: episode: 4, duration: 0.022s, episode steps: 15, steps per second: 695, episode reward: 15.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.600 [0.000, 1.000], mea

 736/1000: episode: 33, duration: 0.217s, episode steps: 23, steps per second: 106, episode reward: 23.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.435 [0.000, 1.000], mean observation: 0.084 [-0.821, 1.438], loss: 0.381029, mean_absolute_error: 1.652247, mean_q: 2.736923
 761/1000: episode: 34, duration: 0.223s, episode steps: 25, steps per second: 112, episode reward: 25.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.400 [0.000, 1.000], mean observation: 0.084 [-1.018, 2.032], loss: 0.397556, mean_absolute_error: 1.702432, mean_q: 2.826024
 774/1000: episode: 35, duration: 0.114s, episode steps: 13, steps per second: 114, episode reward: 13.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.462 [0.000, 1.000], mean observation: 0.098 [-0.965, 1.479], loss: 0.412570, mean_absolute_error: 1.743468, mean_q: 2.908941
 790/1000: episode: 36, duration: 0.153s, episode steps: 16, steps per second: 105, episode reward: 16.000, mean reward: 1.000 [1.000, 1.000], mean act

In [16]:
# DO NOT EDIT

# test
hist = model.test(env, nb_episodes=50, visualize=False)
print('mean:', np.mean(hist.history['episode_reward']))

Testing for 50 episodes ...
Episode 1: reward: 8.000, steps: 8
Episode 2: reward: 10.000, steps: 10
Episode 3: reward: 9.000, steps: 9
Episode 4: reward: 10.000, steps: 10
Episode 5: reward: 8.000, steps: 8
Episode 6: reward: 9.000, steps: 9
Episode 7: reward: 10.000, steps: 10
Episode 8: reward: 10.000, steps: 10
Episode 9: reward: 9.000, steps: 9
Episode 10: reward: 9.000, steps: 9
Episode 11: reward: 9.000, steps: 9
Episode 12: reward: 10.000, steps: 10
Episode 13: reward: 9.000, steps: 9
Episode 14: reward: 9.000, steps: 9
Episode 15: reward: 9.000, steps: 9
Episode 16: reward: 10.000, steps: 10
Episode 17: reward: 10.000, steps: 10
Episode 18: reward: 9.000, steps: 9
Episode 19: reward: 10.000, steps: 10
Episode 20: reward: 10.000, steps: 10
Episode 21: reward: 9.000, steps: 9
Episode 22: reward: 8.000, steps: 8
Episode 23: reward: 8.000, steps: 8
Episode 24: reward: 8.000, steps: 8
Episode 25: reward: 10.000, steps: 10
Episode 26: reward: 10.000, steps: 10
Episode 27: reward: 10.